In [1]:
with open("system_prompt.md", "r") as f:
    system_prompt = f.read()        

In [2]:
import dotenv   
dotenv.load_dotenv()

True

In [3]:
import datasets
ds = datasets.load_dataset("weathon/ava_embeddings", split="train")

In [4]:
ds = ds.with_format("numpy")

In [5]:
import numpy as np

names = ds["name"]
sources = ds["source"]
arr = ds.data.column("embeddings").to_numpy()
arr = np.stack(arr, axis=0)
ava_dataset = ds.filter(lambda example: example["source"] == "ava")
ls_dataset = ds.filter(lambda example: example["source"] == "liminal_space")
lapis_dataset = ds.filter(lambda example: example["source"] == "lapis")
ava_embeddings = np.stack(ava_dataset["embeddings"], axis=0)
ls_embeddings = np.stack(ls_dataset["embeddings"], axis=0)
lapis_embeddings = np.stack(lapis_dataset["embeddings"], axis=0) 

In [6]:
ava_names = ava_dataset["name"]
ls_names = ls_dataset["name"]
lapis_names = lapis_dataset["name"]

In [7]:
import sys
sys.path.append("../")
from qwen3_vl_embedding import Qwen3VLEmbedder

model_name_or_path = "Qwen/Qwen3-VL-Embedding-8B"

model = Qwen3VLEmbedder(model_name_or_path=model_name_or_path, device="cpu", attn_implementation="sdpa")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/749 [00:00<?, ?it/s]

In [ ]:
import nest_asyncio
import os
nest_asyncio.apply()

from agents import Agent, Runner, function_tool, set_default_openai_client, set_tracing_export_api_key

from openai import AsyncOpenAI
import torch

custom_client = AsyncOpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.getenv("OPENROUTER_API_KEY"))
set_default_openai_client(custom_client)

set_tracing_export_api_key(os.getenv("OPENAI_KEY"))

dataset_map = {
    "photos": "ava",
    "dreamcore": "liminal_space",
    "lapis": "lapis"
}
@function_tool
def search(query: str, dataset: str, negative_prompts: list[str] = [], negative_threshold: float = 0.3, t: int = 10) -> str:
    print(f"[LOG] Searching for '{query}' in dataset '{dataset}' with negative prompt '{negative_prompts}' and threshold {negative_threshold} for {t} items...")


    query_texts = [
        {"text": text} for text in negative_prompts
    ]

    image_embeddings = torch.tensor(arr).cpu().float()
    combined_mask = torch.zeros(len(image_embeddings), dtype=torch.bool)

    for q in query_texts:
        q_emb = model.process([q]).cpu().float()
        sim = torch.nn.functional.cosine_similarity(image_embeddings, q_emb)
        combined_mask |= (sim > negative_threshold)

    target_indices = torch.where(combined_mask)[0].tolist()
    empty_images = {names[i] for i in target_indices}
    
    queries = [{"text": query}] 
    query_embedding = model.process(queries).cpu()
    embeddings = (ava_embeddings if dataset == "photos" else ls_embeddings if dataset == "dreamcore" else lapis_embeddings)
    res = torch.nn.functional.cosine_similarity(torch.tensor(embeddings).float(), query_embedding.float())
    selected_images = []
    res = res[0]
    print(res.shape)
    for idx in torch.argsort(res, descending=True):
        if names[idx] not in empty_images:
            selected_images.append(names[idx])
        if len(selected_images) >= t:
            break
    print(f"[LOG] Search results: {selected_images}.")
    return [
        Image.open(f"/home/wg25r/Downloads/ds/train/{dataset_map[dataset]}/{name}") for name in selected_images
    ]


agent = Agent(name="Assistant", 
              tools=[search],
              instructions=system_prompt,
              model="moonshotai/kimi-k2.5")

result = Runner.run_sync(agent, "element: darkness.", max_turns=3)